In [2]:




from gqimax.instructor import Instructor
from gqimax.mapper import map_cx, map_noncx

from gqimax.sample import sample2
from gqimax.mapper import weightsss_to_lambdas
import cupy as cp
import time

def map_noncx(lambdass, indicesss, lut_at_k):
	# k is the index of the U operators, from outside, ranged from 0 to K-1/K'-1
    weightsss = [cp.zeros_like(arr) for arr in indicesss]
    for J, indicess in enumerate(indicesss): # Stabilizer index, from 0 to n - 1
		# Indeciess is a k x n-dim list, example: [IXXX, YYYY] => [[0, 1, 1, 1], [2, 2, 2, 2]]
        # Index is a n-dim list, example: [IXXX] => [[0, 1, 1, 1]]
        for k, indices in enumerate(indicess):
            for j, index in enumerate(indices): # j is the qubit index, from 0 to n - 1
                encoded_pauli = index - 1 # 0 is I, 1 is X, 2 is Y, 3 is Z
                if encoded_pauli == -1:
                    weightsss[J][k][j] = cp.array([1,0,0,0])
                else:
                    weightsss[J][k][j] = lut_at_k[j][encoded_pauli]
	# Lambdass: n x k_i, Indicesss: n x k_i x n
    
    start = time.time()
    lambdass, indicesss = weightsss_to_lambdas(lambdass, weightsss)
    print("Weight to lambda time:", time.time() - start)
    return lambdass, indicesss

cumsum = 0
def map_noncx2(lambdass, indicesss, lut_at_k):
    weightsss = []
    for arr in indicesss:
        H, W = arr.shape
        j_indices = cp.tile(cp.arange(W), (H, 1)) 
        values = arr
        out = cp.empty((H, W, 4), dtype=cp.float32)
        mask0 = (values == 0) # Mask for I
        out[mask0] = cp.array([1, 0, 0, 0], dtype=cp.float32)
        mask123 = ~mask0 # Mask for X,Y,Z
        j_valid = j_indices[mask123]
        val_valid = values[mask123] - 1
        out[mask123] = lut_at_k[j_valid, val_valid]
        weightsss.append(out)
    start = time.time()
    lambdass, indicesss = weightsss_to_lambdas(lambdass, weightsss)
    global cumsum 
    cumsum += time.time() - start
    return lambdass, indicesss


start = time.time()
num_qubits = 4
ins = sample2(num_qubits, 20)
ins.operatoring()
from gqimax.pstabilizers import PStabilizers
stb = PStabilizers(num_qubits)
lambdass = stb.lambdass
indicesss = stb.indicess

for j, order in enumerate(ins.orders):
    k = j // 2
    if order == 0:
        lambdass, indicesss = map_noncx2(lambdass, indicesss, ins.lut[k])
    else:
        for _, indices, _ in ins.xoperators[k]:
            lambdass, indicesss = map_cx(lambdass, indicesss, indices[0], indices[1])
print("Time:", time.time() - start)
print(cumsum)

Map indices to indicess time: 0.0
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0010025501251220703
Map indices to indicess time: 0.0010004043579101562
Map indices to indicess time: 0.0
Map indices to indicess time: 0.001001119613647461
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0010044574737548828
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0009999275207519531
Map indices to indicess time: 0.0
Map indices to indicess time: 0.001001596450805664
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0
Map indices to indicess time: 0.0
Time: 1.5445590019226074
1.453089952468872


In [79]:
ins.lut[0]

array([[[ 1.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 2.        , -0.8337377 , -0.42691758, -0.35017547],
        [ 3.        , -0.36763045,  0.9023816 , -0.22484507]],

       [[ 4.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 0.        , -0.8337377 , -0.42691758, -0.35017547],
        [ 0.        , -0.36763045,  0.9023816 , -0.22484507]],

       [[ 5.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 0.        , -0.8337377 , -0.42691758, -0.35017547],
        [ 0.        , -0.36763045,  0.9023816 , -0.22484507]],

       [[ 0.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 0.        , -0.8337377 , -0.42691758, -0.35017547],
        [ 0.        , -0.36763045,  0.9023816 , -0.22484507]]],
      dtype=float32)

In [77]:

ins.lut[0][2][0][0] = 5

In [78]:
indicesss = [
    cp.array([[0, 3, 2], [1, 1, 1], [1, 1, 1]]),  # shape (4,2)
    cp.array([[1, 1, 3], [2, 0, 1]]),               
	cp.array([[1, 2, 3]]),  
]

map_noncx2(lambdass, indicesss, ins.lut[0])

[array([[[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        , -0.36763045,  0.9023816 , -0.22484507],
        [ 0.        , -0.8337377 , -0.42691758, -0.35017547]],

       [[ 1.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 4.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 5.        ,  0.41198224, -0.05872664, -0.90929747]],

       [[ 1.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 4.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 5.        ,  0.41198224, -0.05872664, -0.90929747]]],
      dtype=float32)]
[array([[[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        , -0.36763045,  0.9023816 , -0.22484507],
        [ 0.        , -0.8337377 , -0.42691758, -0.35017547]],

       [[ 1.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 4.        ,  0.41198224, -0.05872664, -0.90929747],
        [ 5.        ,  0.41198224, -0.05872664, -0.90929747]],

       [[ 1.        ,  0.41198224, -

In [80]:
import cupy as cp

# Dummy LUT for example
LUT = cp.array([
    [[0,1,2,3], [4,5,6,7], [8,9,10,11]],
    [[12,13,14,15], [16,17,18,19], [20,21,22,23]],
    [[24,25,26,27], [28,29,30,31], [32,33,34,35]],
], dtype=cp.float32)

# Ragged tensor
indicess = [
    cp.array([[0, 0, 2], [1, 1, 1], [1, 1, 1], [1, 1, 1]]),  # shape (4,2)
    cp.array([[1, 1, 1], [2, 0, 1]]),                 # shape (2,2)
]

def map_single_arr(arr):
    H, W = arr.shape
    j_indices = cp.tile(cp.arange(W), (H, 1))
    values = arr

    out = cp.empty((H, W, 4), dtype=cp.float32)

    mask0 = (values == 0)
    out[mask0] = cp.array([1, 0, 0, 0], dtype=cp.float32)

    mask123 = ~mask0
    j_valid = j_indices[mask123]
    val_valid = values[mask123] - 1
    out[mask123] = LUT[j_valid, val_valid]

    return out
streams = [cp.cuda.Stream() for _ in indicess]
mapped = [None] * len(indicess)

for i, (arr, stream) in enumerate(zip(indicess, streams)):
    with stream:
        mapped[i] = map_single_arr(arr)

# Đồng bộ tất cả stream
for s in streams:
    s.synchronize()

In [ ]:
import cupy as cp
import random

# Giả lập LUT có 16 cột (j), mỗi j có 3 lớp tra cứu cho giá trị 1,2,3
# Mỗi vector là 4 chiều
# Tăng kích thước LUT: 64 cột j, mỗi j có 3 lớp (cho val = 1,2,3), vector 4 chiều
LUT = cp.random.randn(64, 3, 4).astype(cp.float32)

# Vector cố định cho giá trị 0
vec0 = cp.array([1, 0, 0, 0], dtype=cp.float32)

# Tạo 1000 ragged tensors lớn, mỗi cái shape (512, 64)
def generate_large_ragged_batch(num_arrays=50, height=100000, width=50):
    indicess = []
    for _ in range(num_arrays):
        arr = cp.random.randint(0, 4, size=(height, width), dtype=cp.int32)
        indicess.append(arr)
    return indicess

indicess = generate_large_ragged_batch()
import time

def map_single_arr(arr):
    H, W = arr.shape
    j_indices = cp.tile(cp.arange(W), (H, 1))
    values = arr

    out = cp.empty((H, W, 4), dtype=cp.float32)

    mask0 = (values == 0)
    out[mask0] = vec0

    mask123 = ~mask0
    j_valid = j_indices[mask123]
    val_valid = values[mask123] - 1
    out[mask123] = LUT[j_valid, val_valid]

    return out

# Benchmark tuần tự
start = time.time()
mapped_seq = [map_single_arr(arr) for arr in indicess]
cp.cuda.Device(0).synchronize()
print("⏱ Tuần tự:", time.time() - start)

# Benchmark song song với stream
streams = [cp.cuda.Stream() for _ in indicess]
mapped_parallel = [None] * len(indicess)

start = time.time()
for i, (arr, stream) in enumerate(zip(indicess, streams)):
    with stream:
        mapped_parallel[i] = map_single_arr(arr)

for s in streams:
    s.synchronize()
print("⚡ Song song (stream):", time.time() - start)


⏱ Tuần tự: 10.284321308135986
⚡ Song song (stream): 16.561147689819336


: 